In [1]:
from functools import partial
import sys
sys.path.insert(0, '/Users/wakita/Dropbox (smartnova)/lib/python3/sympy')
#sys.path.insert(0, '/Users/wakita/db-smartnova.bak/lib/python3/sympy')

import sympy as sp
from sympy.utilities.lambdify import lambdify, lambdastr

import numpy as np
np.seterr(all='raise')

from scipy.optimize import minimize

from nbsupport import md
from snsympy import *

In [2]:
from functools import partial
import sys
sys.path.insert(0, '/Users/wakita/Dropbox (smartnova)/lib/python3/sympy')
#sys.path.insert(0, '/Users/wakita/db-smartnova.bak/lib/python3/sympy')

import sympy as sp
from sympy.utilities.lambdify import lambdify, lambdastr

import numpy as np
np.seterr(all='raise')

from scipy.optimize import minimize

from nbsupport import md
from snsympy import *

# D-dimensional KK method

## Dimension

In [3]:
dim = 2

The symbol `dim` represents the dimension of the visualization.  Normally its value is either 2 or 3 but a larger value can be given.

## Graph

Consider a graph $G = (V, E)$ that consists form $n$ vertices ($|V| = n$).

In [4]:
n = sp.Symbol('n', integer=True)

## The spring system

The KK spring model assumes that every pair of vertices $(v_i, v_j)$ is connected by a spring whose natural length and strength are $L_{i, j}$ and $K_{i, j}$, respectively.

KK method picks up a vertex $p$ and attempts to relocate it, while keeping other vertices stationary, trying to minimize the spring potential.  By repeating this process, the spring potential for the whole spring system is minimized.

To keep things simple, we consider springs connected to a vertex $v_i \in V$.  Let $v_j \in P \setminus \{v_i\}$ be a vertex other than $v_i$ in the graph, location function $P: (P \setminus \{v_i\}) \rightarrow R^{\mathrm {dim}}$ denotes the location of the vertex (other than $v_i$), $p$ be the current location of $v_i$, and that $L_j$ and that $K_j$ give the natural length and the strength of the spring that connects $v_i$ and $v_j$.

In [5]:
ni = n-1
Pi = sp.IndexedBase('P', shape=(dim, ni))
Ki = sp.IndexedBase('K')
Li = sp.IndexedBase('L')

p = sp.IndexedBase('p')

j, d = [sp.Idx(*spec) for spec in [('j', ni), ('d', dim)]]
j_range, d_range = [(idx, idx.lower, idx.upper) for idx in [j, d]]

For convenience, we introduce a notation $q(d)$ that represents the $d$'th element of the location of $v_j$, namely $P_{d, j}$.

In [6]:
def q(d): return Pi[d, j]

### Actual length of a spring that connects $v_i$ and $v_j$

In [7]:
LENGTH_J = sp.Symbol('Length_j')

LengthJ = sp.sqrt(sp.Sum((p[d] - q(d)) ** 2, d_range)).doit()

In [8]:
md('The length of the spring that connects $v_i$ and $v_j$ is the distance between $p$ and $q$: $$',
   LENGTH_J, '=', LengthJ, '$$.')

The length of the spring that connects $v_i$ and $v_j$ is the distance between $p$ and $q$: $$Length_{j}=\sqrt{\left(- P_{0,j} + p_{0}\right)^{2} + \left(- P_{1,j} + p_{1}\right)^{2}}$$.

### Potential energy stored in the spring system

In [9]:
POTENTIAL_J, POTENTIAL = sp.var('Potential_j, Potential')

PotentialJ = Ki[j] * (LengthJ - Li[j]) ** 2 / 2
Potential = sp.Sum(PotentialJ, j_range)

In [10]:
md(r'Potential energy for the spring that connects $p \text{ and }q$: $',
   POTENTIAL_J, '=', Ki[j] * (LENGTH_J - Li[j]), '=', PotentialJ, '$')

md('Collective potential energy of all the springs that connect to $p$: $$',
   POTENTIAL, '=', sp.Sum(POTENTIAL_J, j_range), '=', Potential, '$$')

Potential energy for the spring that connects $p \text{ and }q$: $Potential_{j}=\left(Length_{j} - L_{j}\right) K_{j}=\frac{K_{j}}{2} \left(\sqrt{\left(- P_{0,j} + p_{0}\right)^{2} + \left(- P_{1,j} + p_{1}\right)^{2}} - L_{j}\right)^{2}$

Collective potential energy of all the springs that connect to $p$: $$Potential=\sum_{j=0}^{n - 2} Potential_{j}=\sum_{j=0}^{n - 2} \frac{K_{j}}{2} \left(\sqrt{\left(- P_{0,j} + p_{0}\right)^{2} + \left(- P_{1,j} + p_{1}\right)^{2}} - L_{j}\right)^{2}$$

## Code generation

A python function that implements a mathematical formula is obtained by passing the formula to the `lambdify` a list of free variable names and the formula.  The python function is a lambda form that takes actual parameters that correspond to the free variables and (numerically) computes the formula.

Using `lambdify`, we can easily obtain a Python function `f_potential` that computes the spring potential energy ($\mathit {Potential}$).

In [11]:
Params = [n, Ki, Li, Pi, p]

potential_j = lambdify((*Params, j), PotentialJ, dummify=False)
potential = lambdify(Params, Potential, dummify=False)

In [12]:
md('The implementation of the math formula $', PotentialJ, '$ in Python function is the following:')
md('```\n', lambdastr((*Params, j), PotentialJ), '\n```')

md('The implementation of the math formula $', Potential, '$ in Python function is the following:')
md('\n\n```\n', lambdastr(Params, Potential), '\n```')

The implementation of the math formula $\frac{K_{j}}{2} \left(\sqrt{\left(- P_{0,j} + p_{0}\right)^{2} + \left(- P_{1,j} + p_{1}\right)^{2}} - L_{j}\right)^{2}$ in Python function is the following:

```
lambda n,K,L,P,p,j: ((sqrt((-P[0, j] + p[0])**2 + (-P[1, j] + p[1])**2) - L[j])**2*K[j]/2)
```

The implementation of the math formula $\sum_{j=0}^{n - 2} \frac{K_{j}}{2} \left(\sqrt{\left(- P_{0,j} + p_{0}\right)^{2} + \left(- P_{1,j} + p_{1}\right)^{2}} - L_{j}\right)^{2}$ in Python function is the following:



```
lambda n,K,L,P,p: ((builtins.sum((sqrt((-P[0, j] + p[0])**2 + (-P[1, j] + p[1])**2) - L[j])**2*K[j]/2 for j in range(0, n - 2+1))))
```

In [13]:
def testcase(P, K, L, i):
    n = P.shape[1]
    Ki = K[:,i]
    Li = L[:,i]
    p  = P[:,i]
    Pi = P.take(list(range(i)) + list(range(i+1, n)), axis=1)
    return [n, Ki, Li, Pi, p]

def testcase1():
    P = np.eye(dim, dim+1) * 5
    n = P.shape[1]
    K = L = np.ones((n, n))
    return testcase(P, K, L, dim)

def test_potential():
    md('## Tests')
    md('### Potential test')
    args = testcase1()
    Pi = args[3]
    
    for j in range(Pi.shape[1]):
        md('- $', POTENTIAL, '_', j, '=', potential_j(*args, j), '$')

test_potential()

## Tests

### Potential test

- $Potential_0=8.0$

- $Potential_1=8.0$

## Derivative Formulae

In [14]:
X = [p[i] for i in range(dim)]
PotentialDerivatives, potential_derivatives = derivatives(Potential, X, Params)

### Function

In [15]:
md('$$', PotentialDerivatives[0], '$$')

$$\sum_{j=0}^{n - 2} \left(- \sqrt{P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}} L_{j} + \frac{L_{j}^{2}}{2} + \frac{P_{0,j}^{2}}{2} - P_{0,j} p_{0} + \frac{P_{1,j}^{2}}{2} - P_{1,j} p_{1} + \frac{p_{0}^{2}}{2} + \frac{p_{1}^{2}}{2}\right) K_{j}$$

### Gradient

In [16]:
md('$$', PotentialDerivatives[1], '$$')

$$\left[\begin{matrix}\sum_{j=0}^{n - 2} \left(- P_{0,j} + p_{0} + \frac{L_{j} P_{0,j}}{\sqrt{P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}}} - \frac{L_{j} p_{0}}{\sqrt{P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}}}\right) K_{j}\\\sum_{j=0}^{n - 2} \left(- P_{1,j} + p_{1} + \frac{L_{j} P_{1,j}}{\sqrt{P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}}} - \frac{L_{j} p_{1}}{\sqrt{P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}}}\right) K_{j}\end{matrix}\right]$$

### Hessian

In [17]:
md('$$', PotentialDerivatives[2], '$$')

$$\left[\begin{matrix}\sum_{j=0}^{n - 2} \left(1 - \frac{L_{j}}{\sqrt{P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}}} + \frac{L_{j} P_{0,j}^{2}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} - \frac{2 L_{j} P_{0,j} p_{0}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} + \frac{L_{j} p_{0}^{2}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}}\right) K_{j} & \sum_{j=0}^{n - 2} \left(\frac{L_{j} P_{0,j} P_{1,j}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} - \frac{L_{j} P_{0,j} p_{1}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} - \frac{L_{j} P_{1,j} p_{0}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} + \frac{L_{j} p_{0} p_{1}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}}\right) K_{j}\\\sum_{j=0}^{n - 2} \left(\frac{L_{j} P_{0,j} P_{1,j}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} - \frac{L_{j} P_{0,j} p_{1}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} - \frac{L_{j} P_{1,j} p_{0}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} + \frac{L_{j} p_{0} p_{1}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}}\right) K_{j} & \sum_{j=0}^{n - 2} \left(1 - \frac{L_{j}}{\sqrt{P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}}} + \frac{L_{j} P_{1,j}^{2}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} - \frac{2 L_{j} P_{1,j} p_{1}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}} + \frac{L_{j} p_{1}^{2}}{\left(P_{0,j}^{2} - 2 P_{0,j} p_{0} + P_{1,j}^{2} - 2 P_{1,j} p_{1} + p_{0}^{2} + p_{1}^{2}\right)^{\frac{3}{2}}}\right) K_{j}\end{matrix}\right]$$

### Derivatives test

In [18]:
def test_derivatives():
    args = testcase1()
    
    md('- Potential: $', potential_derivatives[0](*args), '$')
    md('- Gradient: $', potential_derivatives[1](*args), '$')
    md('- Hessian: $', potential_derivatives[2](*args), '$')

test_derivatives()

- Potential: $16.0$

- Gradient: $\left[\begin{matrix}-4.0\\-4.0\end{matrix}\right]$

- Hessian: $\left[\begin{matrix}1.8 & 0.0\\0.0 & 1.8\end{matrix}\right]$

### Minimization test

In [19]:
def test_minimize():
    def wrap(f):
        return lambda xs: f(np.array(xs, dtype=np.float))
    
    ni, Ki, Li, Pi, p = testcase1()
    args = [ni, Ki, Li, Pi]
    md('$n_i:', ni, ', K_i:', Ki, ', L_i:', Li, ', P_i:', Pi, ', p:', p, '$')
    
    f, g, h = [wrap(partial(f, *args)) for f in potential_derivatives]
    
    res = minimize(f, p.tolist(), jac=lambda x: g(x).flatten(), hess=h, method='trust-ncg')

    print(res)

test_minimize()

$n_i:3, K_i:\left[\begin{matrix}1.0\\1.0\\1.0\end{matrix}\right], L_i:\left[\begin{matrix}1.0\\1.0\\1.0\end{matrix}\right], P_i:\left[\begin{matrix}5.0 & 0.0\\0.0 & 5.0\end{matrix}\right], p:\left[\begin{matrix}0.0\\0.0\end{matrix}\right]$

     fun: 6.4289321881345245
    hess: array([[ 1.71715729, -0.28284271],
       [-0.28284271,  1.71715729]])
     jac: array([ -3.16141868e-09,  -3.16141868e-09])
 message: 'Optimization terminated successfully.'
    nfev: 5
    nhev: 4
     nit: 4
    njev: 5
  status: 0
 success: True
       x: array([ 2.5,  2.5])
